<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://www.hupi.fr/" ><img src="http://www.hupi.fr/wp-content/uploads/2016/03/hupi_logo_vectoris_menu.png" style="max-width: 300px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# *Text Mining* et Catégorisation des Produits CDiscount avec (py)[Spark](http://spark.apache.org/) et [MLlib](http://spark.apache.org/mllib/)

## 1 Introduction

Il s'agit d'une version simplifiée du concours proposé par Cdiscount et paru sur le site [datascience.net](https://www.datascience.net/fr/challenge). Les données d'apprentissage sont accessibles sur demande auprès de Cdiscount mais les solutions de l'échantillon test du concours ne sont pas et ne seront pas rendues publiques. Un échantillon test est donc construit pour l'usage de ce tutoriel.  L'objectif est de prévoir la catégorie d'un produit à partir de son descriptif. Seule la catégorie principale (1er niveau) est prédite au lieu des trois niveaux demandés dans le concours. L'objectif est plutôt de comparer les performances des méthodes et technologies en fonction de la taille de la base d'apprentissage ainsi que d'illustrer sur un exemple complexe le prétraitement de données textuelles. La stratégie de sous ou sur échantillonnage des catégories qui permet d'améliorer la prévision n'a pas été mise en oeuvre.
* L'exemple est présenté sur un échantillon réduit d'un million de produits au lieu des 15M initiaux
* L'échantillon réduit peut encore l'être puis séparé en 2 parties: apprentissage et validation. 
* Les données textuelles sont  nettoyées, "racinisées", "hashées", "vectorisées" avant modélisation.
* Trois modélisations sont estimées: logistique, arbre, forêt aléatoire.
* Optimiser l'erreur en faisant varier différents paramètres: types et paramètres de vectorisation (hashage, TF-IDF), paramètres de la régression logistique (pénalisation L1) et de la forêt aléatoire (nombre d'arbres et nombre de variables aléatoirement tirées).

Ce calepin utilise l'API pySpark pour traiter des données volumineuses distribuées. Les modélisations sont obtenues à l'aide de la librairie [MLlib](http://spark.apache.org/mllib/).

Un autre [calepin](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Tuto_Notebook_Cdiscount.html) utilise un programme python et la librairie [scikit-learn](http://scikit-learn.org/stable/). L'objectif est de comparer les performances: temps de nettoyage des données et d'apprentissage, erreurs de prévision, dans les deux environnements.

Exécuter le code pour différentes tailles (paramètre tauxTot ci-dessous) de l'échantillon d'apprentissage et comparer les qualités de prévision obtenues. 

In [1]:
# Importation des packages
##Nettoyage
import nltk
import re
##Liste
from numpy import array
##Temps
import time
##LabeledPoint
from pyspark.mllib.regression import LabeledPoint
##SparseVector
from pyspark.mllib.linalg import SparseVector
##Hashage et vectorisation
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
##Regression logistique
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
##Decision Tree
from pyspark.mllib.tree import DecisionTree
##Random Forest
from pyspark.mllib.tree import RandomForest

## 2 Nettoyage des données

### 2.1 Lecture des données

In [2]:
# Création de la base distribuée
data = sc.textFile('/user/remi.verdier/cdiscount/data/training.csv')


### 2.2 Extraction sous-échantillon

In [3]:
# Taux de sous-échantillonnage des données pour tester le programme de préparation
# sur un petit jeu de données
taux_donnees=0.001
data_sans_libelle_original= data.filter(lambda x: x.find("Libelle")==-1)
(data_sans_libelle,data2) = data_sans_libelle_original.randomSplit([taux_donnees,1-taux_donnees])
#data_sans_libelle= data.filter(lambda x: x.find("Libelle")==-1)

rdd=data_sans_libelle.map(lambda line: line.split(";"))


Les instructions précédentes ne sont pas exécutées (exécution paresseuse ou *lazy*). Elles ne le sont que lors d'un traitement explicite.

In [4]:
ts=time.time()
size = rdd.count()
te=time.time()
rt_count = te-ts
print("Comptage prend %d s, pour une taille de %d" %(rt_count, size))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:703)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:702)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:702)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1525)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1449)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1724)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1184)
	at org.apache.spark.SparkContext.stop(SparkCon

### 2.3 Nettoyage

Afin de limiter la dimension de l'espace des variables ou *features* tout en conservant les informations essentielles, il est nécessaire de nettoyer les données en appliquant plusieurs étapes:
* Chaque mot est écrit en minuscule.
* Les termes numériques, de ponctuation et autres symboles sont supprimés.
* 155 mots-courants, et donc non informatif, de la langue française sont supprimés (STOPWORDS). Ex: le, la, du, alors, etc...
* Chaque mot est "racinisé", via la fonction STEMMER.stem de la librairie *nltk*. La racinisation transforme un mot en son radical ou sa racine. Par exemple, les mots: cheval, chevaux, chevalier, chevalerie, chevaucher sont tous remplacés par "cheva".


In [ ]:
# liste des mots à supprimer
STOPWORDS = set(nltk.corpus.stopwords.words('french'))
STEMMER = nltk.stem.SnowballStemmer('french')

def clean_text(text):
    text_lower = text.lower()
    text_clean = re.sub('[^a-z_]', ' ', text_lower)
    tokens = [w for w in text_clean.split() if (len(w)>2) and (w not in STOPWORDS)]
    tokens_stem = [ STEMMER.stem(token) for token in tokens]
    return tokens_stem

In [ ]:
rdd_sans_cat=rdd.map(lambda line: line[3]+" "+line[4])
rdd_cleaned = rdd_sans_cat.map(lambda x : clean_text(x))
ts=time.time()
size = rdd_cleaned.count()
te=time.time()
rt = te-ts
print("Nettoyage prend %d s, pour la taille %d" %(rt, size))


## 3 Préparation des données

### 3.1 Construction des Labels 

Convertir les 47 labels d'origine, qui sont dans un format pas acceptable par les fonctions de MLlib, en des entiers numérotés de 0 a 46 nécessaires à la création de données de type *Labeledpoint*

In [ ]:
ts=time.time()
## Créer une rdd constitué simplement des catégories de l'apprentissage
# Crer un ensemble des labels originaux.
rdd_cat=rdd.map(lambda line: line[0]) 
liste_cat = list(set(rdd_cat.collect()))
nb_classe=len(liste_cat)+1
# Créer un dictionnaire avec en entrée les labels originaux et en 
# sortie les labels numéroté de 0 à 46
cleanCatDic = dict([(k,v) for v,k in enumerate(liste_cat)])
# Appliquer le dictionnaire au labels originaux et crer un rdd 
# avec les nouveaux labels pour chaque ligne
rdd_cat_new = rdd_cat.map(lambda x :cleanCatDic[x])
te=time.time()
rt = te-ts
print("Liste_cat prend %d s, taille %d" %(rt, size))

Concatene la rdd des nouveaux labels, avec la rdd des termes nettoyées.

In [ ]:
rdd_cat_cleaned = rdd_cat_new.zip(rdd_cleaned)

### 3.2 Extraction des deux rdd : *train et test*

In [ ]:
ts=time.time()
taux_test=0.1
(trainingData, testData) = rdd_cat_cleaned.randomSplit([1-taux_test, taux_test])
n_test = testData.count()
te=time.time()
rt = te-ts
print("Split data prend %d s, taille %d" %(rt, size))

Si toutes les données sont préparées, il est possible de sous-échantillonner l'échantillon d'apprentissage pour étudier l'impact de sa taille sur l'erreur de prévision.

### 3.3 Sous-échantillonnage

In [ ]:
tauxEch=1
(selecTrainData, dropData) = trainingData.randomSplit([tauxEch, 1-tauxEch])
n_train = selecTrainData.count()

### 3.4 Construction des *features*

La vectorisation, c'est à dire la construction des features à partir de la liste des mots se fait en 2 étapes:
* Le hashage permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori *n_hash* de *features*. Il repose sur la définition d'une *hash function*, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à *n_hash*) des *features*. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrit par [Weinberger et al. (2009)](https://arxiv.org/pdf/0902.2206.pdf). 

*N.B.* *$h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique*  

* Le *TF-IDF*. Le TF-IDF  permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$  mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$. 

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.


Dans le traitement qui suit, étape de hashage et calcul de $TF$ sont associés dans la même fonction.

# TFI-DF 1 

In [ ]:
ts=time.time()
#Extract Feature of the selecTrainData RDD
selecTrainDataFeature = selecTrainData.map(lambda x : x[1])

#Fit the tf and the idf function on the train data

# Paramètre important qui détermine le nombre de "mots" du dictionnaire
# ou encore le nomnre de variables
n_hash=10
hashingTF = HashingTF(n_hash)
tf_train = hashingTF.transform(selecTrainDataFeature)
# les variables sont les fréquences relatives de ces "mots"
idf_model = IDF().fit(tf_train)
tfidf_train = idf_model.transform(tf_train)
te=time.time()
rt = te-ts
print("Hashage et vectorisation prennent %d s, taille %d" %(rt, n_train))

Le modèle *idf* généré précédement sur la rdd selecTrainData, est appliqué sur l'échantillon test.

In [ ]:
ts=time.time()
#Extract Feature of the selecTrainData RDD
testDataFeature = testData.map(lambda x : x[1])
# Calcul du TF
tf_test = hashingTF.transform(testDataFeature)
# les variables sont les fréquences relatives de ces "mots"
tfidf_test = idf_model.transform(tf_test)
te=time.time()
rt = te-ts
print("Hashage et vectorisation prennent %d s, taille %d" %(rt, n_test))

### 3.5 *Labeledpoint*

Concaténation des labels et des variables pour constituer la base d'apprentissage distribuée.

In [ ]:
labeled_train = selecTrainData.map(lambda x : x[0])
labeledFeature_train = labeled_train.zip(tfidf_train)

labeledPoint_train = labeledFeature_train.map(lambda line: LabeledPoint(line[0],line[1]))
ts=time.time()
n_train = labeledPoint_train.count()
te=time.time()
rt = te-ts
print("Jointure prend %d s, taille %d" %(rt, n_train))

In [ ]:
labeled_test = testData.map(lambda x : x[0])
labeledFeature_test = labeled_test.zip(tf_test)

labeledPoint_test = labeledFeature_test.map(lambda line: LabeledPoint(line[0],line[1]))
ts=time.time()
n_test = labeledPoint_test.count()
te=time.time()
rt = te-ts
print("Jointure prend %d s, taille %d" %(rt, n_test))

## 4 Modélisation et test

### 4.1 Regression logistique

Estimation du modèle, le paramètre de pénalisation est pris par défaut sans optimisation.

In [ ]:
ts=time.time()

model_lrm = LogisticRegressionWithLBFGS.train(labeledPoint_train, iterations=100, initialWeights=None, regParam=0.01, 
                                              regType='l2', intercept=True, corrections=10, tolerance=0.0001, 
                                             validateData=True, numClasses=nb_classe) 
#=> num class donnée par len(list_cat)+1
te=time.time()
rt=te - ts
print("Logistique prend %d s" %(rt))

Estimation de l'erreur sur l'échantillon de test

In [ ]:
labelsAndPreds = labeledPoint_test.map(lambda p: (p.label, model_lrm.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(n_test)
print("Erreur Test = " + str(testErr))

# Test (tfidf2)

In [ ]:
rdd_cat_cleanedFeatures = rdd_cat_cleaned.map(lambda x : x[1])


ts=time.time()
n_hash=10000
hashingTF = HashingTF(n_hash)
tf = hashingTF.transform(rdd_cat_cleanedFeatures)
idf = IDF().fit(tf)
tfidf = idf.transform(tf)
te=time.time()
rt = te-ts
print("Hashage takes %d s, for size %d" %(rt, size))

In [ ]:
labeled_test = rdd_cat_cleaned.map(lambda x : x[0])

rdd_pour_labeled=rdd_cat_new.zip(tfidf)
rdd_labeled=rdd_pour_labeled.map(lambda line: LabeledPoint(line[0],line[1]))
ts=time.time()
size = rdd_labeled.count()
te=time.time()
rt = te-ts
print("Labeled takes %d s, for size %d" %(rt, size))

In [ ]:
taux_valid=0.1
(trainingDataSmall, validationDataSmall) = rdd_labeled.randomSplit([1-taux_valid, taux_valid])

In [ ]:
ts=time.time()

model_lrm2 = LogisticRegressionWithLBFGS.train(trainingDataSmall, iterations=100, initialWeights=None, regParam=0.01, 
                                              regType='l2', intercept=True, corrections=10, tolerance=0.0001, 
                                             validateData=True, numClasses=nb_classe) #=> num class donnée par len(list_cat)+1


te=time.time()
rt=te - ts
print("LR takes %d s" %(rt))

In [ ]:
labelsAndPreds = validationDataSmall.map(lambda p: (p.label, model_lrm2.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(size*taux_valid)
print("Training Error = " + str(trainErr))